# Определение позы человека

### Источники

https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker/python?hl=ru#video

https://www.waveshare.com/wiki/23_Pose_Detection_with_MediaPipe

https://yandex.ru/video/preview/1583404843129035429

https://github.com/google-ai-edge/mediapipe/blob/master/docs/solutions/pose.md

In [98]:
import cv2
import time
import mediapipe as mp
import os

In [99]:
# Настройка MediaPipe Pose
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose(static_image_mode=False,
                   model_complexity=1,
                   enable_segmentation=False,
                   min_detection_confidence=0.5,
                   min_tracking_confidence=0.5)

In [100]:

cap = cv2.VideoCapture('stair.mp4')
ret, frame = cap.read()
if not ret:
    print("Не удалось открыть видео.")
    exit()

H, W, _ = frame.shape
print(frame.shape)




(1280, 720, 3)


In [101]:
cv2.namedWindow("Pose Detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Pose Detection", W // 2, H // 2)

# Настройка видео для записи результата
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('output.avi', fourcc, 20, (W, H))
os.makedirs('video', exist_ok=True)

pTime = 0
idx = 0

step_count = 0
last_step_leg = None  

def get_landmark_style():
        landmark_style = {}
        for i in range(33):  # Всего 33 ключевые точки
            if i in [25, 26, 27, 28]:  # Колени и лодыжки
                landmark_style[i] = mpDraw.DrawingSpec(color=(0, 255, 255), circle_radius=3)
            else:
                landmark_style[i] = mpDraw.DrawingSpec(color=(255, 0, 0), circle_radius=3)
        return landmark_style

while True:
    success, img = cap.read()
    if not success:
        break

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    

    results = pose.process(imgRGB)
    styles = get_landmark_style()
    if results.pose_landmarks:
        mpDraw.draw_landmarks(
            img, 
            results.pose_landmarks, 
            mpPose.POSE_CONNECTIONS, 
            landmark_drawing_spec = styles)
        
    landmarks = results.pose_landmarks.landmark
    # print(landmarks[mpPose.PoseLandmark.LEFT_KNEE].y, landmarks[mpPose.PoseLandmark.RIGHT_KNEE].y)
    left_knee_y = landmarks[mpPose.PoseLandmark.LEFT_KNEE].y * H
    right_knee_y = landmarks[mpPose.PoseLandmark.RIGHT_KNEE].y * H

    left_ankle_y = landmarks[mpPose.PoseLandmark.LEFT_ANKLE].y * H
    right_ankle_y = landmarks[mpPose.PoseLandmark.RIGHT_ANKLE].y * H
    # print(left_knee_y, right_knee_y)

    knee_diff = left_knee_y - right_knee_y
    ankle_diff = left_ankle_y - right_ankle_y


    knee_threshold = 10  
    ankle_threshold = 10  

    current_step_leg = None

    if abs(knee_diff) > knee_threshold and abs(ankle_diff) > ankle_threshold:
        if knee_diff < 0 and ankle_diff < 0:
            current_step_leg = 'left'
        elif knee_diff > 0 and ankle_diff > 0:
            current_step_leg = 'right'

        if current_step_leg != last_step_leg and current_step_leg is not None:
            step_count += 1
            last_step_leg = current_step_leg

    cv2.putText(img, f'Stepping: {current_step_leg}', (10, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)

    cv2.putText(img, f'Step Count: {step_count}', (10, 150),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)


    # Расчёт FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime) if pTime != 0 else 0
    pTime = cTime

    
    cv2.putText(img, f'FPS: {int(fps)}', (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 255), 3)

    cv2.imwrite(f"video/{idx}.png", img)
    out.write(img)
    idx += 1

    display_img = cv2.resize(img, (W // 2, H // 2))
    cv2.imshow("Pose Detection", display_img)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# Завершение
cap.release()
out.release()
cv2.destroyAllWindows()

LEFT_ANKLE (нога слева) — id = 27

RIGHT_ANKLE (нога справа) — id = 28

LEFT_KNEE = 25, 

RIGHT_KNEE = 26